In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from math import sqrt
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.regularizers import l2
from helper import series_to_supervised
from model.cnn import cnn_4layer
from model.mlp import mlp_layer

2024-01-20 22:47:07.656649: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [3]:
dataset = pd.read_csv('../../data/energy_weather.csv', index_col=0)
# https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather

dataset.fillna(0, inplace=True)
data = dataset
data

,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,pressure_Bilbao,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,1035.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,1035.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,1035.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,1033.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,1034.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,1034.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0


In [4]:
data.columns

Index(['price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
      dtype='object')

In [5]:
# plt.hist(data['pollution'], bins=20)

### Preprocess

In [6]:
values = data.values

# specify the number of lag hours
n_hours = 24*3
n_features = data.shape[-1]
k = 12
split1 = 0.7
split2 = 0.85

# frame as supervised learning
reframed = series_to_supervised(values, n_hours, k)
print("reframed.shape:", reframed.shape)

reframed.shape: (34980, 2184)


In [7]:
# split into train and test sets
reframed_values = reframed.values
n_train_hours = int(len(reframed_values)*split1)
n_valid_hours = int(len(reframed_values)*split2)

train = reframed_values[:n_train_hours, :]
val = reframed_values[n_train_hours:n_valid_hours, :]
test = reframed_values[n_valid_hours:, :]


# split into input and outputs
n_obs = n_hours * n_features
feature_idx = 0
train_X, train_y = train[:, :n_obs], train[:, [n_obs + feature_idx + n_features * i for i in range(k)]]
val_X, val_y = val[:, :n_obs], val[:, [n_obs + feature_idx + n_features * i for i in range(k)]]
test_X, test_y = test[:, :n_obs], test[:, [n_obs + feature_idx + n_features * i for i in range(k)]]


print("train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape", 
      train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape
     )

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape (24486, 1872) (24486, 12) (5247, 1872) (5247, 12) (5247, 1872) (5247, 12)


In [8]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))

train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)

val_X = scaler.fit_transform(val_X)
val_y = scaler.fit_transform(val_y)

test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
val_X = val_X.reshape((val_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

print("train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape", 
      train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape
     )

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape (24486, 72, 26) (24486, 12) (5247, 72, 26) (5247, 12) (5247, 72, 26) (5247, 12)


### PM threshold

In [9]:
train_X_pm = train_X[:, 0, feature_idx]
print(train_X_pm.shape)

val_X_pm = val_X[:, 0, feature_idx]
print(val_X_pm.shape)

test_X_pm = test_X[:, 0, feature_idx]
print(test_X_pm.shape)

(24486,)
(5247,)
(5247,)


In [10]:
percentile = 95

merged_array = np.concatenate((train_X_pm, val_X_pm, test_X_pm))

percentile_pm = np.percentile(merged_array, percentile)

print("{}th Percentile of Daily Rain:".format(percentile), percentile_pm)

95th Percentile of Daily Rain: 0.7037105569454176


### train_X_filter

In [11]:
train_X_extreme = train_X[train_X_pm > percentile_pm]
print(train_X_extreme.shape)

train_y_extreme = train_y[train_X_pm > percentile_pm]
print(train_y_extreme.shape)

(696, 72, 26)
(696, 12)


In [12]:
train_X_normal = train_X[train_X_pm <= percentile_pm]
print(train_X_normal.shape)

train_y_normal = train_y[train_X_pm <= percentile_pm]
print(train_y_normal.shape)

(23790, 72, 26)
(23790, 12)


### val_X_filter

In [13]:
val_X_extreme = val_X[val_X_pm > percentile_pm]
print(val_X_extreme.shape)

val_y_extreme = val_y[val_X_pm > percentile_pm]
print(val_y_extreme.shape)

(55, 72, 26)
(55, 12)


In [14]:
val_X_normal = val_X[val_X_pm <= percentile_pm]
print(val_X_normal.shape)

val_y_normal = val_y[val_X_pm <= percentile_pm]
print(val_y_normal.shape)

(5192, 72, 26)
(5192, 12)


### test_X_filter

In [15]:
test_X_extreme = test_X[test_X_pm > percentile_pm]
print(test_X_extreme.shape)

test_y_extreme = test_y[test_X_pm > percentile_pm]
print(test_y_extreme.shape)

(998, 72, 26)
(998, 12)


In [16]:
test_X_normal = test_X[test_X_pm <= percentile_pm]
print(test_X_normal.shape)

test_y_normal = test_y[test_X_pm <= percentile_pm]
print(test_y_normal.shape)

(4249, 72, 26)
(4249, 12)


### Model

In [35]:
# ===== model parameters ======
mlp_unit1 = 128
mlp_unit2 = 128
mlp_unit3 = 64
mlp_unit4 = 64
mlp_unit5 = 32
mlp_unit6 = 32
mlp_unit7 = 16
mlp_unit8 = 16
dropout = 0.0
kernel_size = 2
pool_size = 2
learning_rate = 1e-4
decay_steps = 10000
decay_rate = 0.95
PATIENCE = 50
EPOCHS = 500
BATCH = 512
opt_num = k
input_shape = train_X.shape[1:]

In [36]:
# trained_cnn = load_model('../../saved_models/price_all.h5')
# trained_cnn = load_model('../../saved_models/price_all_weighted_IPF_95.h5')
# trained_cnn = load_model('../../saved_models/price_all_weighted_EVT_95.h5') 
trained_cnn = load_model('../../saved_models/price_all_weighted_META.h5') 

freeze_layer_index = 9

for i, layer in enumerate(trained_cnn.layers):
    #print(i, layer)
    if i <= freeze_layer_index:
        layer.trainable = False


l2_strength = 1e-4
for i in range(freeze_layer_index+1, len(trained_cnn.layers)):
    trained_cnn.layers[i].kernel_regularizer = l2(l2_strength)


trained_cnn._name = 'all_95_ft'
# trained_cnn.summary()

In [21]:
trained_cnn.compile(optimizer='adam',
                    loss='mse',
                    metrics=['mae']
                   )


es = EarlyStopping(monitor='val_mae', mode='min', verbose=2, patience=PATIENCE)
mc = ModelCheckpoint(
#                       '../../saved_models/price_all_weighted_META_95_ft.h5', 
                     '../../saved_models/price_all_weighted_EVT_95_ft.h5', 
#                      '../../saved_models/price_all_weighted_IPF_95_ft.h5', 
#                      '../../saved_models/price_all_95_ft.h5', 
                     monitor='val_mae', 
                     mode='min', 
                     verbose=2, 
                     save_best_only=True,
                    )


trained_cnn.fit(train_X_extreme, train_y_extreme,
                validation_data=(val_X_extreme, val_y_extreme),
                epochs=EPOCHS,
                batch_size=BATCH,
                verbose=2,
                shuffle=True,
                callbacks=[es, mc]
               )

2024-01-19 13:25:14.104750: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-01-19 13:25:14.105549: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1700105000 Hz


Epoch 1/500


2024-01-19 13:25:15.179838: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


2/2 - 3s - loss: 0.0198 - mae: 0.0999 - val_loss: 0.0113 - val_mae: 0.0959

Epoch 00001: val_mae improved from inf to 0.09591, saving model to ../../saved_models/energy_all_weighted_EVT_95_ft.h5
Epoch 2/500
2/2 - 0s - loss: 0.0139 - mae: 0.1029 - val_loss: 0.0262 - val_mae: 0.1485

Epoch 00002: val_mae did not improve from 0.09591
Epoch 3/500
2/2 - 0s - loss: 0.0238 - mae: 0.1393 - val_loss: 0.0052 - val_mae: 0.0564

Epoch 00003: val_mae improved from 0.09591 to 0.05642, saving model to ../../saved_models/energy_all_weighted_EVT_95_ft.h5
Epoch 4/500
2/2 - 0s - loss: 0.0068 - mae: 0.0663 - val_loss: 0.0177 - val_mae: 0.1237

Epoch 00004: val_mae did not improve from 0.05642
Epoch 5/500
2/2 - 0s - loss: 0.0174 - mae: 0.1192 - val_loss: 0.0073 - val_mae: 0.0720

Epoch 00005: val_mae did not improve from 0.05642
Epoch 6/500
2/2 - 0s - loss: 0.0062 - mae: 0.0641 - val_loss: 0.0071 - val_mae: 0.0633

Epoch 00006: val_mae did not improve from 0.05642
Epoch 7/500
2/2 - 0s - loss: 0.0080 - mae:

Epoch 54/500
2/2 - 0s - loss: 0.0021 - mae: 0.0358 - val_loss: 0.0039 - val_mae: 0.0457

Epoch 00054: val_mae did not improve from 0.04226
Epoch 55/500
2/2 - 0s - loss: 0.0021 - mae: 0.0352 - val_loss: 0.0038 - val_mae: 0.0444

Epoch 00055: val_mae did not improve from 0.04226
Epoch 56/500
2/2 - 0s - loss: 0.0021 - mae: 0.0352 - val_loss: 0.0038 - val_mae: 0.0457

Epoch 00056: val_mae did not improve from 0.04226
Epoch 57/500
2/2 - 0s - loss: 0.0021 - mae: 0.0349 - val_loss: 0.0039 - val_mae: 0.0463

Epoch 00057: val_mae did not improve from 0.04226
Epoch 58/500
2/2 - 0s - loss: 0.0021 - mae: 0.0348 - val_loss: 0.0039 - val_mae: 0.0455

Epoch 00058: val_mae did not improve from 0.04226
Epoch 59/500
2/2 - 0s - loss: 0.0020 - mae: 0.0344 - val_loss: 0.0039 - val_mae: 0.0467

Epoch 00059: val_mae did not improve from 0.04226
Epoch 60/500
2/2 - 0s - loss: 0.0020 - mae: 0.0343 - val_loss: 0.0040 - val_mae: 0.0473

Epoch 00060: val_mae did not improve from 0.04226
Epoch 61/500
2/2 - 0s - los

2/2 - 0s - loss: 0.0013 - mae: 0.0274 - val_loss: 0.0044 - val_mae: 0.0502

Epoch 00113: val_mae did not improve from 0.04226
Epoch 114/500
2/2 - 0s - loss: 0.0013 - mae: 0.0273 - val_loss: 0.0050 - val_mae: 0.0545

Epoch 00114: val_mae did not improve from 0.04226
Epoch 115/500
2/2 - 0s - loss: 0.0013 - mae: 0.0278 - val_loss: 0.0043 - val_mae: 0.0500

Epoch 00115: val_mae did not improve from 0.04226
Epoch 116/500
2/2 - 0s - loss: 0.0013 - mae: 0.0273 - val_loss: 0.0048 - val_mae: 0.0534

Epoch 00116: val_mae did not improve from 0.04226
Epoch 117/500
2/2 - 0s - loss: 0.0012 - mae: 0.0271 - val_loss: 0.0044 - val_mae: 0.0501

Epoch 00117: val_mae did not improve from 0.04226
Epoch 118/500
2/2 - 0s - loss: 0.0013 - mae: 0.0272 - val_loss: 0.0046 - val_mae: 0.0523

Epoch 00118: val_mae did not improve from 0.04226
Epoch 119/500
2/2 - 0s - loss: 0.0012 - mae: 0.0268 - val_loss: 0.0045 - val_mae: 0.0516

Epoch 00119: val_mae did not improve from 0.04226
Epoch 120/500
2/2 - 0s - loss: 0.0

2/2 - 0s - loss: 7.5678e-04 - mae: 0.0214 - val_loss: 0.0055 - val_mae: 0.0566

Epoch 00171: val_mae did not improve from 0.04226
Epoch 172/500
2/2 - 0s - loss: 7.4872e-04 - mae: 0.0212 - val_loss: 0.0057 - val_mae: 0.0571

Epoch 00172: val_mae did not improve from 0.04226
Epoch 173/500
2/2 - 0s - loss: 7.5001e-04 - mae: 0.0212 - val_loss: 0.0055 - val_mae: 0.0558

Epoch 00173: val_mae did not improve from 0.04226
Epoch 174/500
2/2 - 0s - loss: 7.3729e-04 - mae: 0.0211 - val_loss: 0.0059 - val_mae: 0.0585

Epoch 00174: val_mae did not improve from 0.04226
Epoch 175/500
2/2 - 0s - loss: 7.5742e-04 - mae: 0.0214 - val_loss: 0.0058 - val_mae: 0.0577

Epoch 00175: val_mae did not improve from 0.04226
Epoch 176/500
2/2 - 0s - loss: 7.2754e-04 - mae: 0.0209 - val_loss: 0.0055 - val_mae: 0.0557

Epoch 00176: val_mae did not improve from 0.04226
Epoch 177/500
2/2 - 0s - loss: 7.2543e-04 - mae: 0.0209 - val_loss: 0.0062 - val_mae: 0.0599

Epoch 00177: val_mae did not improve from 0.04226
Epoch 

2/2 - 0s - loss: 5.5281e-04 - mae: 0.0183 - val_loss: 0.0063 - val_mae: 0.0603

Epoch 00228: val_mae did not improve from 0.04226
Epoch 229/500
2/2 - 0s - loss: 5.1568e-04 - mae: 0.0177 - val_loss: 0.0069 - val_mae: 0.0637

Epoch 00229: val_mae did not improve from 0.04226
Epoch 230/500
2/2 - 0s - loss: 5.1383e-04 - mae: 0.0176 - val_loss: 0.0064 - val_mae: 0.0606

Epoch 00230: val_mae did not improve from 0.04226
Epoch 231/500
2/2 - 0s - loss: 5.0892e-04 - mae: 0.0176 - val_loss: 0.0063 - val_mae: 0.0603

Epoch 00231: val_mae did not improve from 0.04226
Epoch 232/500
2/2 - 0s - loss: 5.1075e-04 - mae: 0.0176 - val_loss: 0.0070 - val_mae: 0.0641

Epoch 00232: val_mae did not improve from 0.04226
Epoch 233/500
2/2 - 0s - loss: 5.0191e-04 - mae: 0.0175 - val_loss: 0.0066 - val_mae: 0.0621

Epoch 00233: val_mae did not improve from 0.04226
Epoch 234/500
2/2 - 0s - loss: 4.8534e-04 - mae: 0.0171 - val_loss: 0.0065 - val_mae: 0.0617

Epoch 00234: val_mae did not improve from 0.04226
Epoch 